In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../')
import velovae as vv

In [2]:
dataset = 'Pancreas'
import scvelo as scv
root = "/scratch/blaauw_root/blaauw1/gyichen"
#adata = scv.datasets.pancreas()
adata = anndata.read_h5ad(f'{root}/data/{dataset}_pp.h5ad')

In [ ]:
# Uncomment this if data is not preprocessed
#vv.preprocess(adata, n_gene=2000, min_shared_counts=20)
#adata.write_h5ad(f'data/{dataset}_pp.h5ad')

In [3]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'data/velovae/continuous/{dataset}'
gene_plot = ['Cpe','Gng12', 'Ppp3ca', 'Smoc1']

# VeloVAE

In [4]:
figure_path = f'{figure_path_base}/VeloVAE'
model_path = f'{model_path_base}/VeloVAE'

torch.manual_seed(2022)
np.random.seed(2022)

vae = vv.VAE(adata, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0')

vae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

vae.save_model(model_path, 'encoder', 'decoder')
vae.save_anndata(adata, 'velovae', data_path, file_name=f'{dataset}.h5ad')

Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Detected 901 velocity genes.
Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

3 clusters detected based on gene co-expression.
(0.55, 0.317384117041346), (0.45, 0.7672515333922927)
(0.57, 0.8124603340211373), (0.43, 0.44375671501048647)
(0.49, 0.34814455909332653), (0.51, 0.7329919503301096)
KS-test result: [0. 0. 0.]
Initial induction: 1164, repression: 836/2000
Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


...

Percentage of Invalid Sets: 0.027
Average Set Size: 96
Finished. Actual Time:   0 h :  0 m :  3 s
*********       Round 4: Early Stop Triggered at epoch 1076.       *********
Change in noise variance: 0.0
Change in x0: 0.07335051594349523
*********             Velocity Refinement Round 5              *********
Cell-wise KNN Estimation.


  0%|          | 0/3696 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.027
Average Set Size: 96
Finished. Actual Time:   0 h :  0 m :  3 s
*********       Round 5: Early Stop Triggered at epoch 1107.       *********
Change in noise variance: 0.0
Change in x0: 0.058973681733413684
*********             Velocity Refinement Round 6              *********
Cell-wise KNN Estimation.


  0%|          | 0/3696 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.027
Average Set Size: 96
Finished. Actual Time:   0 h :  0 m :  3 s
*********       Round 6: Early Stop Triggered at epoch 1176.       *********
Change in noise variance: 0.0
Change in x0: 0.03721050404529367
*********             Velocity Refinement Round 7              *********
Stage 2: Early Stop Triggered at round 6.
*********              Finished. Total Time =   0 h :  6 m : 23 s             *********
Final: Train ELBO = 5067.038,	Test ELBO = 5048.854


# Full VB

In [6]:
figure_path = f'{figure_path_base}/FullVB'
model_path = f'{model_path_base}/FullVB'

torch.manual_seed(2022)
np.random.seed(2022)

full_vb = vv.VAE(adata, 
                 tmax=20, 
                 dim_z=5, 
                 device='cuda:0',
                 full_vb=True)

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'fullvb', data_path, file_name=f'{dataset}.h5ad')

Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Train a Branching ODE

In [ ]:
figure_path = f'{figure_path_base}/BrODE'
model_path = f'{model_path_base}/BrODE'

torch.manual_seed(2022)
np.random.seed(2022)
brode = vv.BrODE(adata, 'clusters', 'fullvb_time', 'fullvb_z')

brode.print_weight()
brode.train(adata, 'fullvb_time', 'clusters', plot=False, gene_plot=gene_plot, figure_path=figure_path)

brode.print_weight()

brode.save_model(model_path, 'brode')
brode.save_anndata(adata, 'brode', data_path, file_name=f'{dataset}.h5ad')

vv.plot_transition_graph(adata, save=f'{figure_path_base}/transition.png')

# Evaluation

In [ ]:
cluster_edges = [('Ngn3 high EP', 'Pre-endocrine'),
                 ('Pre-endocrine', 'Delta'), 
                 ('Pre-endocrine', 'Beta'), 
                 ('Pre-endocrine','Epsilon'), 
                 ('Pre-endocrine','Alpha')]
vv.post_analysis(adata,
                 'eval',
                 ['VeloVAE', 'FullVB', 'BrODE'],
                 ['velovae', 'fullvb', 'brode'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)